In [5]:
# switch to TF environment
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Dense, LSTM, Masking, TimeDistributed, Input
from tensorflow.keras import backend as K
import scipy.io as sci
import h5py
import numpy as np
import csv as csv
import math
import os
from scipy import stats
from scipy.signal import find_peaks
import tensorflow as tf

In [ ]:
# Essential info about the input and output filepaths

N_EXP = 12 # number of experiments
N_FOVS = 30 # number of fields of view per experiment

# We only to track 2 in this example
track = 2

# Raw trajectory Dataset Filepath (input)
path_trajectoryDat = "ref/"

# Results filepath for saving
path_results = ''
path_track = path_results + f'track_{track}/'

In [6]:
dimension = 2 # spatial dimension 
minSeg = 3 # minimum segment length allowed
padval = -99. # padding value that is masked by the nets
maxLen = 200 # maximum trajectory length allowed

In [7]:
# run to define the loss functions for the models below
def masked_mse(y_true, y_pred, padval=-99.):
    a_output = y_pred[:, :, 0]
    K_output = y_pred[:, :, 1]
    true_a = y_true[:, :, 0]
    true_K = y_true[:, :, 1]

    # MSE for alpha
    mask = K.cast(K.not_equal(true_a, padval), dtype=a_output.dtype)
    squared_diff = K.square(true_a - a_output)
    masked_squared_diff = squared_diff * mask

    #MSLE for K
    true_K=true_K*mask
    K_output=K_output*mask
    log_diff = K.square(K.log(true_K+1) - K.log(K_output+1))
    masked_log_diff = log_diff * mask

    # Summing and averaging over the non-padded timesteps
    num_non_padding = K.sum(mask)
    mse = K.sum(masked_squared_diff) / (num_non_padding + K.epsilon())  + 2*K.sum(masked_log_diff) / (num_non_padding + K.epsilon())
    return mse

def masked_mae(y_true, y_pred): # masked mean absolute error
    padval=-99.
    float_output = y_pred
    true_float = y_true

    mask = K.cast(K.not_equal(true_float, padval), dtype=float_output.dtype)
    squared_diff = K.cast(K.abs(true_float - float_output), dtype=float_output.dtype)
    masked_squared_diff = squared_diff * mask
    # Summing and averaging over the non-padded timesteps
    num_non_padding = K.sum(mask)
    mse = K.sum(masked_squared_diff) / (num_non_padding + K.epsilon())  # Adding epsilon to avoid division by zero
    return mse

def OrdEnt(y_true, y_pred): # an *ordinal* sparse categorical crossentropy
    maxInt = 3.
    padval=int(-99.)
    numCP_true = tf.cast(y_true, dtype=tf.int32)
    numCP_pred = y_pred

    mask = tf.cast(tf.not_equal(numCP_true, padval), dtype=numCP_true.dtype)
    numCP_true=numCP_true*mask # cant have -99 in the label for cross ent

    numCP_loss = K.sparse_categorical_crossentropy(numCP_true, numCP_pred)
    mask=tf.cast(mask, dtype=numCP_loss.dtype)
    numCP_loss = numCP_loss * mask
    weights = K.abs(K.cast(numCP_true, dtype=tf.float32) - K.cast(K.argmax(numCP_pred, axis=-1),dtype=tf.float32))/maxInt

    return tf.reduce_sum((1.0+weights*mask)*numCP_loss) / tf.cast(tf.reduce_sum(mask), dtype=numCP_loss.dtype)

def maskedBin(y_true, y_pred): # masked binary entropy
    padval=int(-99.)
    numCP_true = tf.cast(y_true, dtype=tf.int32)
    numCP_pred = y_pred

    mask = tf.cast(tf.not_equal(numCP_true, padval), dtype=numCP_true.dtype)
    numCP_true=numCP_true*mask # cant have -99 in the label for cross ent

    numCP_loss = K.binary_crossentropy(tf.cast(numCP_true, dtype=numCP_pred.dtype), numCP_pred)
    numCP_loss = numCP_loss*tf.cast(mask, dtype=numCP_loss.dtype)

    return tf.reduce_sum(numCP_loss) / tf.cast(tf.reduce_sum(mask), dtype=numCP_loss.dtype)

def masked_categorical_crossentropy(y_true, y_pred): # masked categorical cross entropy for multiclass OHE label
    one_hot_output = y_pred[:, :, :4]
    true_one_hot = y_true[:, :, :4]
    padval=-99.
    mask = tf.reduce_any(tf.not_equal(true_one_hot, padval), axis=-1)
    mask = tf.cast(mask, true_one_hot.dtype)
    
    loss = tf.keras.losses.categorical_crossentropy(true_one_hot, one_hot_output, from_logits=False)
    loss = loss * mask # float 32vs64
    
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def int_acc(y_true, y_pred): # masked accuracy from sparse (integer) labels
    padval=int(-99.)
    numCP_true = tf.cast(y_true, tf.int64)
    numCP_pred = y_pred

    mask = tf.not_equal(numCP_true, padval)
    correct_predictions = tf.equal(numCP_true, tf.argmax(numCP_pred, axis=-1))
    acc = tf.reduce_sum(tf.cast(correct_predictions, tf.float32) * tf.cast(mask, tf.float32)) / tf.reduce_sum(tf.cast(mask, tf.float32))
    return acc

def one_hot_accuracy(y_true, y_pred): # masked accuracy from one hot encoded labels
    padval=-99.
    true_labels = y_true[:, :, :4] # 4 class - one hot encoding
    pred_labels = y_pred[:, :, :4]
    mask = tf.reduce_all(tf.not_equal(true_labels, padval), axis=-1)
    correct_predictions = tf.equal(tf.argmax(true_labels, axis=-1), tf.argmax(pred_labels, axis=-1))
    accuracy = tf.reduce_sum(tf.cast(correct_predictions, tf.float32) * tf.cast(mask, tf.float32)) / tf.reduce_sum(tf.cast(mask, tf.float32))
    return accuracy


In [8]:
def buildArchitectureCPtOnly(minSeg, padval, dimension): # defines network for model to predict where changepoints occur
    block_size = minSeg*(dimension+1)                                   # Size of the blocks of data points

    input_shape = (None, block_size)  
    inputs = Input(shape=input_shape)
    masked_inputs = Masking(mask_value=padval)(inputs) # masking layer with specific padding value

    lstm_out = LSTM(250,                              # first layer: LSTM of dimension 250
                    return_sequences=True,            # return sequences for the second LSTM layer            
                    recurrent_dropout=0.2,            # recurrent dropout for preventing overtraining
                    input_shape=(None, block_size))(masked_inputs)
    # but LSTM input is 3D, this is added on as batch size later? should batches have same length trajectories or just blocks constant
                                                            
    lstm2_out = LSTM(50,                              # second layer: LSTM of dimension 50
                    return_sequences=True,            # also returns sequences, into the TimeDistributed
                    recurrent_dropout=0.2,           
                    dropout=0)(lstm_out)
    
    yn_output_layer = TimeDistributed(Dense(1, activation='sigmoid'), name='yn_output')(lstm2_out) # 1 binary prediction per sequence point
    OH_output_layer = TimeDistributed(Dense(4, activation='softmax'), name='OH_output')(lstm2_out) # ordinal classification for exact changepoint location in block
    model_inference = Model(inputs=inputs, outputs=[yn_output_layer,OH_output_layer])
    model_inference.compile(optimizer='adam', 
                            loss={
                                  'yn_output':maskedBin,
                                  'OH_output':OrdEnt}, 
                            metrics={
                                     'yn_output':'accuracy',
                                     'OH_output':int_acc}, 
                            loss_weights={
                                  'yn_output':1.,
                                  'OH_output':1.})
    
    print(model_inference.summary())

    return model_inference

model_CPtOnly = buildArchitectureCPtOnly(minSeg, padval, dimension)

model_CPtOnly.load_weights('DynamicCP_Weights.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, 9)]    0           []                               
                                                                                                  
 masking_1 (Masking)            (None, None, 9)      0           ['input_2[0][0]']                
                                                                                                  
 lstm_2 (LSTM)                  (None, None, 250)    260000      ['masking_1[0][0]']              
                                                                                                  
 lstm_3 (LSTM)                  (None, None, 50)     60200       ['lstm_2[0][0]']                 
                                                                                            

In [9]:
def buildArchitectureCPtKandA(minSeg, padval, dimension): # defines the network for the model predicting changepoints, K and alpha
    
    block_size = minSeg*(dimension+1)                               

    input_shape = (None, block_size)  
    inputs = Input(shape=input_shape)
    masked_inputs = Masking(mask_value=padval)(inputs)

    lstm_out = LSTM(250,                            
                    return_sequences=True,                      
                    recurrent_dropout=0.2,            
                    input_shape=(None, block_size))(masked_inputs)
                                                            
    lstm2_out = LSTM(50,                              
                    return_sequences=True,          
                    recurrent_dropout=0.2,           
                    dropout=0)(lstm_out)
    
    fl_output_layer = TimeDistributed(Dense(2), name='fl_output')(lstm2_out) # 2 floats: alpha and K, for regression on every point in sequence
    yn_output_layer = TimeDistributed(Dense(1, activation='sigmoid'), name='yn_output')(lstm2_out)
    OH_output_layer = TimeDistributed(Dense(4, activation='softmax'), name='OH_output')(lstm2_out)
    model_inference = Model(inputs=inputs, outputs=[fl_output_layer,yn_output_layer,OH_output_layer])
    model_inference.compile(optimizer='adam', 
                            loss={'fl_output':masked_mse,
                                  'yn_output':maskedBin,
                                  'OH_output':OrdEnt}, 
                            metrics={'fl_output':masked_mae,
                                     'fl_output':masked_mse,
                                     'yn_output':'accuracy',
                                     'OH_output':int_acc}, 
                            loss_weights={'fl_output':1.,
                                  'yn_output':1.,
                                  'OH_output':1.})
    
    print(model_inference.summary())

    return model_inference

model_CPtKanda = buildArchitectureCPtKandA(minSeg, padval, dimension)

model_CPtKanda.load_weights('DynamicCP_KandA_Weights.h5')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 9)]    0           []                               
                                                                                                  
 masking_2 (Masking)            (None, None, 9)      0           ['input_3[0][0]']                
                                                                                                  
 lstm_4 (LSTM)                  (None, None, 250)    260000      ['masking_2[0][0]']              
                                                                                                  
 lstm_5 (LSTM)                  (None, None, 50)     60200       ['lstm_4[0][0]']                 
                                                                                            

In [10]:
def buildArchitectureModelKandA(minSeg, padval, dimension): # model predicts the time-varying diffustion type, K and alpha

    block_size = minSeg*(dimension+1)

    input_shape = (None, block_size)  
    inputs = Input(shape=input_shape)
    masked_inputs = Masking(mask_value=padval)(inputs)

    lstm_out = LSTM(250,                            
                    return_sequences=True,                      
                    recurrent_dropout=0.2,            
                    input_shape=(None, block_size))(masked_inputs)
                                                            
    lstm2_out = LSTM(50,                              
                    return_sequences=True,          
                    recurrent_dropout=0.2,           
                    dropout=0)(lstm_out)
    
    float_output_layer = TimeDistributed(Dense(2), name='float_output')(lstm2_out) # 2 floats, K and alpha
    one_hot_output_layer = TimeDistributed(Dense(4, activation='softmax'), name='one_hot_output')(lstm2_out) # 4 different diffusion classes/models

    model_inference = Model(inputs=inputs, outputs=[float_output_layer, one_hot_output_layer])
    model_inference.compile(optimizer='adam', 
                            loss={'float_output':masked_mse,
                                  'one_hot_output':masked_categorical_crossentropy}, 
                            metrics={'float_output':masked_mae,
                                     'one_hot_output':one_hot_accuracy},
                            loss_weights={
                                  'float_output':1.,
                                  'one_hot_output':2.})
    print(model_inference.summary())

    return model_inference

model_Modelt = buildArchitectureModelKandA(minSeg, padval, dimension)

model_Modelt.load_weights('DynamicDiffType_KandA_Weights.h5')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, 9)]    0           []                               
                                                                                                  
 masking_3 (Masking)            (None, None, 9)      0           ['input_4[0][0]']                
                                                                                                  
 lstm_6 (LSTM)                  (None, None, 250)    260000      ['masking_3[0][0]']              
                                                                                                  
 lstm_7 (LSTM)                  (None, None, 50)     60200       ['lstm_6[0][0]']                 
                                                                                            

In [11]:
# Data Preparation; retrieves *increments* from the positions, normalises them, and pads the trajectories to accomodate the blocks.
def data_prepare(X,N,dimension,blocksize,padval,maxLen):                 
    thr=1e-10
    r = [np.diff(xi,axis=1) for xi in X] # taking the increments along each dimension of each trajectory
    sxy = [np.std(ri.flatten()) for ri in r] # the std of both x and y increments of each trajectory
    meanxy = [np.mean(ri.flatten()) for ri in r] # the mean of both x and y increments of each trajectory

    # maxLen=(maxLen-1)//minSeg + 1
    maxLen = (maxLen-1)//minSeg + (1 if (maxLen-1)%minSeg != 0 else 0)
    datLen = int(blocksize/minSeg) # size of the vector for each position.
    
    x = []
    for i, ri in enumerate(r): # each iteration a different traj
        trajlen=len(ri[0])
        y=ri
        mu=np.full((dimension,1),meanxy[i])
        sig=np.full((dimension,1),sxy[i])
        y = (y-mu) / np.where(sxy[i]>thr,sig,1)   # len(y) == dimension
        y = np.array([[y[0][n], y[1][n],sxy[i]] for n in range(trajlen)]) # stack on the std of xy
        y=np.transpose(y,axes=[1,0])
        if (trajlen%minSeg) != 0:
            y = np.concatenate((y,np.full((datLen,minSeg-(trajlen%minSeg)),padval)),axis=1) 

        padding = maxLen-int(datLen*len(y[0])/blocksize) # This padding does not affect the predictions, but having all trajectories be the same length helps to process together in arrays.
        if  padding > 0:
            y = np.concatenate((y,np.full((datLen,padding*minSeg),padval)),axis=1)

        y=np.transpose(y,axes=[1,0])
        y=y.reshape(int(datLen*(len(y))/blocksize),blocksize)
        x.append(np.array(y))
    
    return x

In [17]:
X = [[] for _ in range(N_EXP)]
trfoIdx = [[] for _ in range(N_EXP)]
for exp in range(N_EXP):
    for fov in range(N_FOVS):
        with open(path_trajectoryDat+'track_2/'+"exp_"+str(exp)+"/trajs_fov_"+str(fov)+'.csv','r') as csv_file: # just change this path to the dataset you want predictions for.
            csv_reader = csv.reader(csv_file)
            line_count = 0
            traj = []
            t = 0
            for row in csv_reader:
                if line_count == 0:
                    # print(f'Column names are {", ".join(row)}')
                    line_count += 1
                else:
                    if float(row[0]) == t:
                        traj.append([float(row[2]),float(row[3])]) # just keep X and Y
                    else: 
                        X[exp].append(traj)
                        t += 1
                        traj = []
                        traj.append([float(row[2]),float(row[3])])
            t += 1
            trfoIdx[exp].append(t)
            X[exp].append(traj) # last trajectory in the fov
X = np.array(X)
trajlens=[len(X[i][j]) for i in range(N_EXP) for j in range(len(X[i]))]

FileNotFoundError: [Errno 2] No such file or directory: 'track_2/exp_0/trajs_fov_0.csv'

In [14]:
sortedX = [[] for _ in range(N_EXP)]
for exp in range(N_EXP):
    for i in range(len(X[exp])):
        sortedX[exp].append(list(np.array(X[exp][i])[:,0])+list(np.array(X[exp][i])[:,1]))
dimsortX=[[np.array(i).reshape(dimension,-1) for i in x] for x in sortedX]


In [ ]:
block_size = minSeg*(dimension+1)
Xin = [data_prepare(dimsortX[i],len(dimsortX[i]),dimension,block_size,padval, maxLen) for i in range(N_EXP)] # Prepare the data for the nets
Xin2 = [data_prepare([x[:,1:] for x in dimsortX[i]],len(dimsortX[i]),dimension,block_size,padval, maxLen) for i in range(N_EXP)] # Prepare it again, shifting (with a cut) by 1 point
Xin3 = [data_prepare([x[:,2:] for x in dimsortX[i]],len(dimsortX[i]),dimension,block_size,padval, maxLen) for i in range(N_EXP)] # shift by 2 points 

In [ ]:
modtPred = [model_Modelt.predict(np.array(x)) for x in Xin]
# modtPred2 = [model_Modelt.predict(np.array(x)) for x in Xin2]
# modtPred3 = [model_Modelt.predict(np.array(x)) for x in Xin3]
ModT = [np.argmax(e[1],axis=-1) for e in modtPred]

In [ ]:
predCPtKanda = [model_CPtKanda.predict(np.array(x)) for x in Xin]
predCPtKanda2 = [model_CPtKanda.predict(np.array(x)) for x in Xin2]
predCPtKanda3 = [model_CPtKanda.predict(np.array(x)) for x in Xin3]

In [ ]:
cptPred = [model_CPtOnly.predict(np.array(x)) for x in Xin]
cptPred2 = [model_CPtOnly.predict(np.array(x)) for x in Xin2]
cptPred3 = [model_CPtOnly.predict(np.array(x)) for x in Xin3]

In [ ]:
def roundThresh(arr, thr=0.5): # you may change the threhsold for rounding the binary prediction if you wish
    newarr=[]
    for d in arr:
        if d >= thr:
            d = -(d // -1)
        newarr.append(d // 1)
    return np.array(newarr).astype(int)

In [ ]:
# The Sliding Window
alphaMeansPred = [[] for _ in range(N_EXP)]
KMeansPred = [[] for _ in range(N_EXP)]
CPWinPred = [[] for _ in range(N_EXP)] # changepoint storage
for e in range(N_EXP):
    for t, traj in enumerate(dimsortX[e]):
        incLen = len(traj[0])-1 # number of increments 
        coarseLen=incLen//minSeg + (1 if incLen%minSeg != 0 else 0) # length of "coarse-grained" increment sequence input to the nets

        # extract the length of useful (unpadded) sequence in the in/output of the nets, for original (Xin) and shifted (Xin2, Xin3) trajectories
        check = len([p for p in Xin[e][t][:,0] if p != padval]) 
        check2 = len([p for p in Xin2[e][t][:,0] if p != padval]) 
        check3 = len([p for p in Xin3[e][t][:,0] if p != padval])
        if check != coarseLen:
            print('AH')

        alphaWindowsPred = [[] for _ in range(incLen)] # tracking the value of alpha over the fully discretised time
        KWindowsPred = [[] for _ in range(incLen)] # tracking the value of K over the fully discretised time
        cpsTrajPred = np.zeros(incLen) # for tracking changepoint detections over the discretised time

        predBinwin = roundThresh(cptPred[e][0][t][:,0])[:coarseLen] # extract binary CP detections from unshifted
        predBinwin2 = roundThresh(cptPred2[e][0][t][:,0])[:check2] # extract binary CP detections from trajectory shifted by 1
        predBinwin3 = roundThresh(cptPred3[e][0][t][:,0])[:check3] # extract binary CP detections from trajectory shifted by 2
        predIntwin = np.argmax(cptPred[e][1][t], axis=-1)[:coarseLen] # extract precise block-CP detections
        predIntwin2 = np.argmax(cptPred2[e][1][t], axis=-1)[:check2] 
        predIntwin3 = np.argmax(cptPred3[e][1][t], axis=-1)[:check3]

        for i in range(incLen):
            alphaWindowsPred[i].append(predCPtKanda[e][0][t][:, 0][i//minSeg]) # taking coarse grained alpha predictions to the complete-time array
            KWindowsPred[i].append(predCPtKanda[e][0][t][:, 1][i//minSeg]) # K
            cpsTrajPred[i] += predBinwin[i//minSeg] # record the binary classification predictions
        for i in range(incLen-1): # for the shifted trajectory
            alphaWindowsPred[i+1].append(predCPtKanda2[e][0][t][:, 0][i//minSeg])
            KWindowsPred[i+1].append(predCPtKanda2[e][0][t][:, 1][i//minSeg])
            cpsTrajPred[i+1] += predBinwin2[i//minSeg]
        for i in range(incLen-2): # for the twice shifted trajectory
            alphaWindowsPred[i+2].append(predCPtKanda3[e][0][t][:, 0][i//minSeg])
            KWindowsPred[i+2].append(predCPtKanda3[e][0][t][:, 1][i//minSeg])
            cpsTrajPred[i+2] += predBinwin3[i//minSeg] 

        for idx, cp in enumerate(predIntwin): # recording the ordinal classification predictions 
            if cp != 0:
                cpsTrajPred[idx*minSeg + cp - 1] += 1 # add one back in (shift right) for real CP, not the increment version this is (-1)
        for idx, cp in enumerate(predIntwin2):
            if cp != 0:
                cpsTrajPred[1+idx*minSeg + cp - 1] += 1
        for idx, cp in enumerate(predIntwin3):
            if cp != 0:
                cpsTrajPred[2+idx*minSeg + cp - 1] += 1

        alphaMeansPred[e].append([np.mean(a) for a in alphaWindowsPred])
        KMeansPred[e].append([np.mean(a) for a in KWindowsPred])
        CPWinPred[e].append(find_peaks(cpsTrajPred,height=2,distance=minSeg)[0] + 1) # noting the prominent detections from cpsTrajPred "histogram", note +1 put back
        

In [ ]:
numCPsWin = [np.array([len(CPWinPred[e][t]) for t in range(len(CPWinPred[e]))]) for e in range(N_EXP)] # number of changepoints detected per trajectory

In [ ]:
import os
for e in range(N_EXP):
    # directory = 'public_data_challenge_v0/res/track_2/exp_'+str(e)
    directory = path_track + 'exp_'+str(e)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
path_track

In [ ]:
# Saving our predictions

h, h1, h2, h3=0, 0, 0, 0
for e in range(N_EXP):    
    path_exp = path_track + f'exp_{e}/'
    t = 0
    for fov in range(N_FOVS):
        submission_file = path_exp + f'fov_{fov}.txt'
        with open(submission_file, 'w') as f:
            for idx in range(trfoIdx[e][fov]):
                prediction_traj = []
                length = len(X[e][t])
                if len(CPWinPred[e][t]) == 0: # if no changepoint detected in the trajectory
                    Ks = np.mean(KMeansPred[e][t]) # take average K(t) value of entire trajectory
                    if Ks < 0:
                        Ks = 0.0001
                    As = np.mean(alphaMeansPred[e][t])
                    if As < 0:
                        As = 0.
                    if e==2 and As < 0.38: # only for experiment 02
                        As = 0.
                        Ks= 0.0001
                    Mods = int(stats.mode(ModT[e][t][:int(-(length // -3))]).mode) # diffustion type just given by the mode
                    # print(diffType)
                    prediction_traj = [idx,
                                       Ks, 
                                       As, 
                                       Mods,
                                       length]

                elif len(CPWinPred[e][t]) >0: # if changepoints present
                    prediction_traj = [idx]
                    cp2= 1
                    # print(e, t)
                    for t_id, time in enumerate(list(CPWinPred[e][t])+[length]): # must have traj.length at the end to calculate last segment
                        h+=1
                        cp = time
                        if cp > length: 
                            h2+=1
                            continue
                        if cp2 > cp:
                            h3+=1
                            continue

                        Ks = np.mean(KMeansPred[e][t][(cp2-1):(cp-1)]) # bc list is of real CPs not increments which need -1
                        if Ks < 0:
                            Ks = 0.0001

                        As = np.mean(alphaMeansPred[e][t][(cp2-1):(cp-1)])
                        if As < 0:
                            As = 0.
                        if e==2 and As < 0.38: # specific to experiment 02 trapped datapoints.
                            As = 0.
                            Ks= 0.0001
                        if math.isnan(As):
                            print(e, t, cp)
                        prediction_traj.append(Ks)
                        prediction_traj.append(As)

                        Mods = ModT[e][t][(cp2-1)//3:(cp-1)//3]
                        if len(Mods) == 0:
                            Mods = ModT[e][t][(cp-1)//3]
                        else:
                            Mods = stats.mode(Mods).mode
                            Mods = int(Mods)

                        prediction_traj.append(Mods)

                        prediction_traj.append(cp)

                        cp2 = np.copy(cp) # keep track of latest changepoint, for the next segment iteration.

                else:
                    print('no')
                t += 1
                formatted_numbers = ','.join(map(str, prediction_traj))
                f.write(formatted_numbers + '\n')
          
print(h,h2,h3)    
                